In [1]:

!pip install turfpy
!pip install geojson
!pip install meteostat
!pip install numpy==1.26.4

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.5 MB/s eta 0:00:0000:0100:01
  Created wheel for turfpy: filename=turfpy-0.0.7-py3-none-any.whl size=39101 sha256=9177d6ddc4351409ef7dfda1fb2f8465ea9f228fdca45ce97d3266a21dc77aff
  Stored in directory: /root/.cache/pip/wheels/f7/f8/07/965ea3fe9ce3d94e9ee6815425f294500adc31eb4c14037c61
Successfully built turfpy
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1752, in print
    extend(render(renderable, render_options))
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console.py", line 1390, in render
    for render_output in iter_render:
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/loggin

In [4]:

import json
import time
import socket
import uuid
import random
from datetime import datetime, timedelta
import pytz
from meteostat import Point, Hourly

with open('sale_points.json', 'r') as f:
    sale_points = json.load(f)
    
# Función para obtener la temperatura actual
def get_current_temperature(latitude, longitude):
    location = Point(latitude, longitude)
    start = datetime.now() - timedelta(hours=1)
    end = datetime.now()
    data = Hourly(location, start, end)
    data = data.fetch()
    temperature = data['temp'].mean() if not data.empty else 20  # Usar 20 como valor predeterminado si no hay datos
    return round(temperature, 1)

def generate_quantity_products(temperature):
    temp_min, temp_max = 10, 30
    normalized_temp = (temperature - temp_min) / (temp_max - temp_min)
    min_quantity, max_quantity = 1, 50
    mean_quantity = normalized_temp * (max_quantity - min_quantity) + min_quantity
    quantity = int(random.gauss(mean_quantity,10))
    return max(min(quantity, max_quantity), min_quantity)

# Generar datos simulados
def generate_event():
    point = random.choice(sale_points)
    date_now = datetime.now(pytz.timezone('America/Bogota'))
    temperature = get_current_temperature(point["latitude"], point["longitude"])
    event = {
        "latitude": point["latitude"],
        "longitude": point["longitude"],
        "date": date_now.strftime("%d/%m/%Y %H:%M:%S"),
        "customer_id": random.randint(1000, 9000),
        "employee_id": point["employees"]["0"] if date_now.weekday() < 5 else point["employees"]["1"],
        "sale_point_id" : point["id"],
        "quantity_products": generate_quantity_products(temperature),
        "order_id": str(uuid.uuid4()),
        "commune": point["commune"],
        "neighborhood": point["neighborhood"],
        "event_day": date_now.day,
        "event_hour": date_now.hour,
        "event_minute": date_now.minute,
        "event_month": date_now.month,
        "event_second": date_now.second,
        "event_year": date_now.year,
        "current_temperature": temperature
    }
    return event

def send_data(s, conn, data):
    conn.sendall(json.dumps(data).encode() + b'\n')
    print("enviado:", data)
    

def generate_data(host, port, interval):

    s = socket.socket()
    s.bind((host, port))
    s.listen(1)
    print(f"Listening on {host}:{port}")

    conn, addr = s.accept()
    print(f"Connection from {addr}")
    try:
        while True:
            event = generate_event()
            # Esperar 30 segundos antes de generar el siguiente evento
            send_data(s, conn, event)
            # Ejemplo de guardado en un archivo JSON
            with open('simulated_sales.json', 'a') as file:
                file.write(json.dumps(event) + "\n")
            
            time.sleep(interval)
            
    except KeyboardInterrupt:
        print("Deteniendo el servidor...")
        conn.close()
        s.close()
        print("Conexión cerrada.")
    
    except BrokenPipeError:
        print("La conexión ha sido cerrada.")
        conn.close()
        s.close()



In [8]:
# generate_data(host, port, interval_in_seconds)
generate_data('localhost', 4225, 1)

Listening on localhost:4225
Connection from ('127.0.0.1', 54792)
enviado: {'latitude': 6.243897, 'longitude': -75.608053, 'date': '17/06/2024 17:50:40', 'customer_id': 5803, 'employee_id': 'Employee_57_w', 'sale_point_id': 57, 'quantity_products': 22, 'order_id': 'b16df0ff-f2e2-44fb-b3e6-e80a401e6479', 'commune': 'Comuna 12 - La América', 'neighborhood': 'Santa Teresita', 'event_day': 17, 'event_hour': 17, 'event_minute': 50, 'event_month': 6, 'event_second': 40, 'event_year': 2024, 'current_temperature': 24.0}
enviado: {'latitude': 6.19411, 'longitude': -75.564301, 'date': '17/06/2024 17:50:41', 'customer_id': 3900, 'employee_id': 'Employee_24_w', 'sale_point_id': 24, 'quantity_products': 43, 'order_id': 'cc2f38ef-56c0-40b1-8538-dcb36b137973', 'commune': 'Comuna 14 - El Poblado', 'neighborhood': 'Los Naranjos', 'event_day': 17, 'event_hour': 17, 'event_minute': 50, 'event_month': 6, 'event_second': 41, 'event_year': 2024, 'current_temperature': 24.0}
enviado: {'latitude': 6.304213, 'l